Задача программы - найти аномалии в почтовых доменах и выделить их в отдельную базу

input:
1) Путь к zip архиву 
2) Пароль от архива (в случае отсутствия пустая строка)

output:
1) .txt файл со строками содержащими домены, встречающиеся реже всех остальных

Комментарии:
Главная трудность, с которой я столкнулся - это медленность python. Переписать программу на Си, у меня не получилось. Пришлось не делать унивирсальность для разных форматов, а ограничится лишь txt файлами.
Ну то что выводятся только домены с минимальными встречами в файле условность, конечно, можно добавть и те домены, которые встречаюися на 2-м месте, считаю в данном случае это неважно.

Алггоритм программы:
1. Перебираем txt файлы в архиве (в каждом сразу же перебираем строки в них)
2. Считаем количество каждого почтового домена (считаем доменом)
3. Когда закончили перебор, начинаем перебирать заного и добавляя в отдельную базу те строки, домены в которых встречаются реже всего

In [ ]:
import zipfile
from collections import Counter
import re

In [ ]:
zip_path = 'b.zip'  # путь к архиву
password = ''       # Пароль
output_file = 'output.txt'

In [ ]:
def open_zip(zip_path, password):
    all_emails = []  # Список для хранения всех найденных электронных почт
    
    with zipfile.ZipFile(zip_path) as z:
        z.setpassword(password.encode())
        for txt_file in z.namelist():
            if txt_file.endswith('.txt'):  # Проверяем, является ли файл текстовым
                with z.open(txt_file) as f:
                    content = f.read().decode('utf-8')  # Читаем содержимое файла
                    # Находим все электронные почты с помощью регулярного выражения
                    emails = re.findall(r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}', content)
                    all_emails.extend(emails)  # Добавляем найденные почты в общий список
    
    return all_emails  # Возвращаем список всех электронных почт

def L_domains(emails):
    domains = [email.split('@')[1] for email in emails] # Извлекаем домены из электронных почт
    domain_counts = Counter(domains)
    
    min_count = min(domain_counts.values())
    least_common_domains = {domain for domain, count in domain_counts.items() if count == min_count} # Самые редкие домены
    
    return least_common_domains  # Возвращаем набор редких доменов

In [ ]:
def main(zip_path, password, output_file):
    # Извлекаем все электронные почты из ZIP-архива
    all_emails = open_zip(zip_path, password)
    # Получаем редкие домены
    least_common_domains = L_domains(all_emails)
    # Фильтруем электронные почты по редким доменам
    least_common_emails = [email for email in all_emails if email.split('@')[1] in least_common_domains]

    # Сохраняем результат в текстовый файл
    with open(output_file, 'w') as f:
        for email in least_common_emails:
            f.write(email + '\n')  # Записываем каждую редкую электронную почту в файл
main(zip_path, password, output_file)